In [ ]:
from src.data_structure import VisitData, Record, RecordKey, record_key_from_abbreviation
from src.database import read_sql

In [ ]:
data = VisitData()

In [ ]:
df = read_sql('''
WITH observations AS (
    SELECT * FROM student_data.vwd_observations
    -- LIMIT 100000
),
cases as (
    SELECT f.visit_oid, f.startingvisitoid
    FROM student_data.vwd_diagnosen AS d JOIN student_data.vwd_faelle AS f ON d.visit_oid = f.visit_oid
    WHERE d."isPrimary" = FALSE
    AND d."Code" ilike('j18%')
)

SELECT
    cases.startingvisitoid AS visit_id,
    observations."Value" AS value,
    observations."StartDT_minutes_since_start" AS minutes_since_start,
    observations."FindingAbbr" AS finding
FROM cases JOIN observations ON cases.visit_oid = observations.patientvisit_oid
ORDER BY minutes_since_start
''')

In [ ]:
def process_row(visit_id, value, minutes_since_start, finding):
    key = record_key_from_abbreviation(finding)
    if not key: return
    record = Record(minutes_since_start, value)
    data.append(visit_id, key, record)

_ = [process_row(*row) for row in zip(*(df[col] for col in df))]

In [ ]:
print(len(data.visits))